In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime

''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
from collections import Counter
import pandas as pd
import numpy as np

'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, normalisation, evaluation, Neptune_data_read
from parameter import *
from dspike_formulas import *

import pylab as plt

/Users/matthias/anaconda/lib/python2.7/site-packages/numexpr/cpuinfo.py:76: UserWarning: [Errno 2] No such file or directory
  stacklevel=stacklevel + 1):


In [4]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2017-09-13_DS_calib/"
files_start = 8248
files_end = 8249

In [41]:
# cup configuration
#cup_config = cycle_Sb
Sn_isotopes = ["117", "118", "119", "120", "122", "124"]
cup_config = cycles_spike

# Mass Range of cup configuration
mass_range = cycle_Sn_spike_mass_range

# Isotopes used for Interference correction
#corr_isotopes_1 = {"Cd" : "111", "Te" : "125"}
corr_isotopes_2 = {"Te" : "125", "Xe" : "129"}
#corr_isotopes_2 = {}
corr_isotopes_Sb = {"Te": "125"}

norm_ratio_int = ["122","118"]
norm_ratio_Sb = ["123", "121"]
denom_isotope_int = norm_ratio_int[1]

Sn_monitor = ["125", "129"]
Sn_monitor = []


def eval_iso_list(isotopes_list,norm_ratio, monitor_iso):
    isotope_nom = norm_ratio[0]
    isotope_den = norm_ratio[1]
    isotopes_list.remove(norm_ratio[1])
    for i in range(len(monitor_iso)):
        isotopes_list.append(monitor_iso[i])
    isotopes_list.sort()
    return isotopes_list

isotope_ls = eval_iso_list(Sn_isotopes, norm_ratio_int, Sn_monitor)

data_sample_column = [(i + "/" + denom_isotope_int) for i in isotope_ls]
isotopes = [isotope_ls]

#number of iterations for beta
iter_beta = 10

n_GPL = -0.15
law = "GPL"

# Interference_corr on the denominator isotope
isotope_denom_corr = False
# background correction
blk_corr = False

In [6]:
isotopes

[['117', '119', '120', '122', '124']]

In [7]:
True_123Sb_121Sb = 0.7479
mod_123Sb_121Sb = 0.747491
mod_122_117Sn = 0.806799155732

In [8]:
columns_1 = ["cycle", "sample", "date", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)", "L4 (1)"]
df_all = pd.DataFrame()
blk_ls = []
sample_ls = []
files_1 = range(files_start, files_end, 1)
#files_1 = range(8195,8218,1) + range(8220,8245, 1) 

for sample in files_1:
    
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    df_zero = df.data_zero_corr(sample)
    df_zero_1 = pd.DataFrame(df_zero["cycle1"])
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    df_zero_1["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    df_zero_1["sample"] = sample_name
    df_zero_1["cycle"] = df_zero_1.index.values
    df_zero_1.index = df_zero_1["date"]
    df_zero_1 = df_zero_1[columns_1]
    df_all = df_all.append(df_zero_1, ignore_index = True)
    
    if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean" or sample_name == "Teflon blk":
        blk_ls.append(sample)
    elif sample_name == "SQ" or sample_name == "Teflon blk" or sample_name == "4ml HDPE blank":
        None
    else:
        sample_ls.append(sample)

df_all.set_index("date", inplace = True)

#for x in [123]:
#    sample_ls.remove(x)
#blk_ls = blk_ls[0::5]
#blk_ls.remove(5234)

In [9]:
df_all

,cycle,sample,H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),Ax (1),L1 (1),L2 (1),L3 (1),L4 (1)
date,,,,,,,,,,,,,,,
2017-09-15 11:51:00,1,NIST 1ppm,0.000497,0.000649,0.000219,-0.000012,1.059480e-05,3.136626,4.272553,2.448095,5.574003,16.796705,4.368148,12.160210,3.800555
2017-09-15 11:51:00,2,NIST 1ppm,0.000503,0.000652,0.000215,-0.000025,1.073660e-05,3.129587,4.261198,2.442266,5.558970,16.757268,4.357930,12.130675,3.791628
2017-09-15 11:51:00,3,NIST 1ppm,0.000522,0.000655,0.000213,-0.000015,2.629370e-05,3.135541,4.267692,2.446955,5.567456,16.789048,4.366101,12.153634,3.798514
2017-09-15 11:51:00,4,NIST 1ppm,0.000507,0.000632,0.000218,-0.000016,5.199000e-07,3.134587,4.268562,2.446134,5.568741,16.783585,4.364775,12.150235,3.797533
2017-09-15 11:51:00,5,NIST 1ppm,0.000486,0.000650,0.000216,-0.000010,1.698760e-05,3.136701,4.267534,2.447878,5.566898,16.795191,4.367391,12.157845,3.799977
2017-09-15 11:51:00,6,NIST 1ppm,0.000486,0.000631,0.000230,-0.000003,1.490000e-08,3.126762,4.256347,2.440216,5.552262,16.741530,4.353529,12.119156,3.787814
2017-09-15 11:51:00,7,NIST 1ppm,0.000506,0.000635,0.000221,-0.000016,6.039200e-06,3.138158,4.273787,2.448891,5.575325,16.802985,4.369667,12.163726,3.801984
2017-09-15 11:51:00,8,NIST 1ppm,0.000511,0.000648,0.000208,-0.000022,2.303820e-05,3.128739,4.260295,2.441780,5.557690,16.754212,4.356970,12.128223,3.790724
2017-09-15 11:51:00,9,NIST 1ppm,0.000510,0.000631,0.000227,-0.000012,2.920300e-06,3.137763,4.274800,2.448749,5.576541,16.802248,4.369650,12.163371,3.801901


In [10]:
database["Te"]["Masses"].get_all_Isotope_masses()

{'120': 119.904021,
 '122': 121.903055,
 '124': 123.902825,
 '125': 124.904435,
 '126': 125.90331,
 '128': 127.904461519,
 '130': 129.906222921}

In [11]:
df_all.groupby(df_all.index).agg([np.mean])

,cycle,H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),Ax (1),L1 (1),L2 (1),L3 (1),L4 (1)
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
date,,,,,,,,,,,,,,
2017-09-15 11:51:00,10.5,0.000502,0.000642,0.000222,-0.000011,0.000014,3.140935,4.277192,2.451252,5.579764,16.818592,4.373827,12.175394,3.805474


In [42]:
files_1 = range(files_start, files_end, 1)
df_raw_signals = pd.DataFrame()
df_internal_norm = pd.DataFrame()
df_Sb_norm = pd.DataFrame()

for sample in sample_ls:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    
     # arbitrary blank positions
    #blk1 = [item for item in blk_ls if item < sample]
    #blk2 = [item for item in blk_ls if item > sample]
    #blk1 = blk1[-1]
    #blk2 = blk2[0]
    #blk_corr_sample = True
    
    if (blk_corr == True) and (blk_corr_sample == True):
        blk_1 = NU_data_read(path, blk1, cup_config)
        blk_2 = NU_data_read(path, blk2, cup_config)
        df_zero = df.data_zero_corr(sample)
        df_bgd_1 = blk_1.data_zero_corr(blk1)
        df_bgd_2 = blk_2.data_zero_corr(blk2)   
        
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio, law, n_GPL)
        df_bgd_1 = new_corr.mad_outlier_rejection_dict(df_bgd_1)
        df_bgd_2 = new_corr.mad_outlier_rejection_dict(df_bgd_2)
        df_bgd_corr = new_corr.data_bgd_corr(df_bgd_1, df_bgd_2)
        #new_corr.line2_corr(df_bgd_corr, "119")
    else:
        df_zero = df.data_zero_corr(sample)
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio, law, n_GPL)
        #new_corr.line2_corr(df_zero, "119")
    
    
    data_int = new_corr.internal_norm_1(norm_ratio_int, denom_isotope_int, iter_beta)
    beta_int = new_corr.beta("Sn", norm_ratio_int, iter_beta)
    data_int = pd.DataFrame.from_dict(data_int, orient = 'index')
    beta_int = pd.Series(beta_int, name="beta_int")
    data_int = new_corr.mad_outlier_rejection(data_int)
    
    #data_Sb = new_corr.external_norm_Sb(norm_ratio_Sb, denom_isotope_int, iter_beta)
    #beta_Sb = new_corr.beta("Sb", norm_ratio_Sb, iter_beta)
    #data_Sb = pd.DataFrame.from_dict(data_Sb, orient = 'index')
    #beta_Sb= pd.Series(beta_Sb, name="beta_Sb")
    #data_Sb = new_corr.mad_outlier_rejection(data_Sb)
   
    
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    
    #data_int = data_int.drop(denom_isotope_int, axis=1)
    data_int = data_int[sorted(data_int.columns)]
    data_int.columns = data_sample_column
    data_int["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_int["sample"] = sample_name
    data_int["cycle"] = data_int.index.values
    data_int = pd.concat((data_int, beta_int), axis=1, join='inner')
    
    #data_Sb = data_Sb.drop(denom_isotope_int, axis=1)
    #data_Sb = data_Sb[sorted(data_Sb.columns)]
    #data_Sb.columns = data_sample_column
    #data_Sb["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    #data_Sb["sample"] = sample_name
    #data_Sb["cycle"] = data_Sb.index.values
    #data_Sb= pd.concat((data_Sb, beta_Sb), axis=1, join='inner')

    df_internal_norm = df_internal_norm.append(data_int)
    #df_Sb_norm = df_Sb_norm.append(data_Sb)
df_internal_norm.set_index("date", inplace = True)
#df_Sb_norm.set_index("date", inplace = True)

In [43]:
df_internal_norm

,117/118,119/118,120/118,122/118,124/118,sample,cycle,beta_int
date,,,,,,,,
2017-09-15 11:51:00,0.316667,0.354566,1.345968,NaN,0.239001,NIST 1ppm,1,1.337018e+09
2017-09-15 11:51:00,0.316697,0.354593,1.346045,0.19125,0.239028,NIST 1ppm,2,1.363514e+09
2017-09-15 11:51:00,0.316674,0.354585,1.346033,NaN,0.239021,NIST 1ppm,3,1.377506e+09
2017-09-15 11:51:00,0.316677,0.354581,1.346007,0.19125,0.239035,NIST 1ppm,4,1.348334e+09
2017-09-15 11:51:00,0.316689,0.354564,1.346037,0.19125,0.239016,NIST 1ppm,5,1.395737e+09
2017-09-15 11:51:00,0.316688,0.354562,1.345983,NaN,0.239000,NIST 1ppm,6,1.427370e+09
2017-09-15 11:51:00,0.316698,0.354583,1.346058,0.19125,0.239036,NIST 1ppm,7,1.357073e+09
2017-09-15 11:51:00,0.316685,0.354587,1.346071,NaN,0.239011,NIST 1ppm,8,1.364272e+09
2017-09-15 11:51:00,0.316698,0.354595,1.346057,NaN,0.239023,NIST 1ppm,9,1.341448e+09


In [ ]:
sample = "NIST 1ppm"
ratio = "120/118"

In [ ]:
plt.plot(df_Sb_norm[df_Sb_norm["sample"] == sample].index, df_Sb_norm[df_Sb_norm["sample"] == sample][ratio], linestyle='-', marker='o', color='b')
plt.show()

In [ ]:
plt.plot(df_internal_norm[df_internal_norm["sample"] == sample].index, df_internal_norm[df_internal_norm["sample"] == sample][ratio], linestyle='-', marker='o', color='b')
plt.show()

In [ ]:
df_Sb_norm["124/118"].mean() - df_internal_norm["124/118"].mean()

In [ ]:
df_Sb_norm_new = pd.DataFrame()

for sample in sample_ls:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    
     # arbitrary blank positions
    blk1 = [item for item in blk_ls if item < sample]
    blk2 = [item for item in blk_ls if item > sample]
    blk1 = blk1[-1]
    blk2 = blk2[0]
    blk_corr_sample = True
    
    if (blk_corr == True) and (blk_corr_sample == True):
        blk_1 = NU_data_read(path, blk1, cup_config)
        blk_2 = NU_data_read(path, blk2, cup_config)
        df_zero = df.data_zero_corr(sample)
        df_bgd_1 = blk_1.data_zero_corr(blk1)
        df_bgd_2 = blk_2.data_zero_corr(blk2)   
        
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
        df_bgd_1 = new_corr.mad_outlier_rejection_dict(df_bgd_1)
        df_bgd_2 = new_corr.mad_outlier_rejection_dict(df_bgd_2)
        df_bgd_corr = new_corr.data_bgd_corr(df_bgd_1, df_bgd_2)
        #new_corr.line2_corr(df_bgd_corr, "119")
    else:
        df_zero = df.data_zero_corr(sample)
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
        #new_corr.line2_corr(df_zero, "119")
    
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
        
    data_Sb_new = new_corr.external_norm_Sb_change_true(norm_ratio_Sb, denom_isotope_int, iter_beta, mod_123Sb_121Sb)
    data_Sb_new = pd.DataFrame.from_dict(data_Sb_new, orient = 'index')
    data_Sb_new = new_corr.mad_outlier_rejection(data_Sb_new)
        
    data_Sb_new = data_Sb_new[sorted(data_Sb_new.columns)]
    data_Sb_new.columns = data_sample_column
    data_Sb_new["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_Sb_new["sample"] = sample_name
    data_Sb_new["cycle"] = data_Sb_new.index.values
        
    df_Sb_norm_new = df_Sb_norm_new.append(data_Sb_new)
    
df_Sb_norm_new.set_index("date", inplace = True)

In [ ]:
df_Sb_norm_new_117 = pd.DataFrame()
data_sample_column_117 = ['118/117', '119/117', '120/117', '122/117', '124/117', '125/117', '129/117']
isotopes_117 = [['118', '119', '120', '122', '124', '125', '129']]

for sample in sample_ls:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    
     # arbitrary blank positions
    blk1 = [item for item in blk_ls if item < sample]
    blk2 = [item for item in blk_ls if item > sample]
    blk1 = blk1[-1]
    blk2 = blk2[0]
    blk_corr_sample = True
    
    if (blk_corr == True) and (blk_corr_sample == True):
        blk_1 = NU_data_read(path, blk1, cup_config)
        blk_2 = NU_data_read(path, blk2, cup_config)
        df_zero = df.data_zero_corr(sample)
        df_bgd_1 = blk_1.data_zero_corr(blk1)
        df_bgd_2 = blk_2.data_zero_corr(blk2)   
        
        new_corr = evaluation(df_zero, cycles, isotopes_117, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
        df_bgd_1 = new_corr.mad_outlier_rejection_dict(df_bgd_1)
        df_bgd_2 = new_corr.mad_outlier_rejection_dict(df_bgd_2)
        df_bgd_corr = new_corr.data_bgd_corr(df_bgd_1, df_bgd_2)
        #new_corr.line2_corr(df_bgd_corr, "119")
    else:
        df_zero = df.data_zero_corr(sample)
        new_corr = evaluation(df_zero, cycles, isotopes_117, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
        #new_corr.line2_corr(df_zero, "119")
    
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
        
    data_Sb_new_117 = new_corr.external_norm_Sb_change_true(norm_ratio_Sb, "117", iter_beta, mod_123Sb_121Sb)
    data_Sb_new_117 = pd.DataFrame.from_dict(data_Sb_new_117, orient = 'index')
    data_Sb_new_117 = new_corr.mad_outlier_rejection(data_Sb_new_117)
        
    data_Sb_new_117 = data_Sb_new_117[sorted(data_Sb_new_117.columns)]
    data_Sb_new_117.columns = data_sample_column_117
    data_Sb_new_117["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_Sb_new_117["sample"] = sample_name
    data_Sb_new_117["cycle"] = data_Sb_new_117.index.values
        
    df_Sb_norm_new_117 = df_Sb_norm_new_117.append(data_Sb_new_117)
    
df_Sb_norm_new_117.set_index("date", inplace = True)


In [ ]:
df_int_norm_new_117 = pd.DataFrame()
data_sample_column_117 = ['118/117', '119/117', '120/117', '122/117', '124/117', '125/117', '129/117']
isotopes_117 = [['118', '119', '120', '122', '124', '125', '129']]

for sample in sample_ls:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    
     # arbitrary blank positions
    blk1 = [item for item in blk_ls if item < sample]
    blk2 = [item for item in blk_ls if item > sample]
    blk1 = blk1[-1]
    blk2 = blk2[0]
    blk_corr_sample = True
    
    if (blk_corr == True) and (blk_corr_sample == True):
        blk_1 = NU_data_read(path, blk1, cup_config)
        blk_2 = NU_data_read(path, blk2, cup_config)
        df_zero = df.data_zero_corr(sample)
        df_bgd_1 = blk_1.data_zero_corr(blk1)
        df_bgd_2 = blk_2.data_zero_corr(blk2)   
        
        new_corr = evaluation(df_zero, cycles, isotopes_117, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
        df_bgd_1 = new_corr.mad_outlier_rejection_dict(df_bgd_1)
        df_bgd_2 = new_corr.mad_outlier_rejection_dict(df_bgd_2)
        df_bgd_corr = new_corr.data_bgd_corr(df_bgd_1, df_bgd_2)
        #new_corr.line2_corr(df_bgd_corr, "119")
    else:
        df_zero = df.data_zero_corr(sample)
        new_corr = evaluation(df_zero, cycles, isotopes_117, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
        #new_corr.line2_corr(df_zero, "119")
    
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
        
    data_int_new_117 = new_corr.internal_norm_Sb_change_true(norm_ratio_int, "117", iter_beta, mod_122_117Sn)
    data_int_new_117 = pd.DataFrame.from_dict(data_int_new_117, orient = 'index')
    #data_int_new_117 = new_corr.mad_outlier_rejection(data_int_new_117)
        
    data_int_new_117 = data_int_new_117[sorted(data_int_new_117.columns)]
    data_int_new_117.columns = data_sample_column_117
    data_int_new_117["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_int_new_117["sample"] = sample_name
    data_int_new_117["cycle"] = data_int_new_117.index.values
        
    df_int_norm_new_117 = df_int_norm_new_117.append(data_int_new_117)
    
df_int_norm_new_117.set_index("date", inplace = True)

In [ ]:
def SE(x):
    SE = np.std(x) / x.count()
    return SE

def SD2(x):
    SD2 = 2 * np.std(x)
    return SD2

df_Sb_norm_sub_mean = df_Sb_norm.drop("cycle", axis=1).groupby(df_Sb_norm.index).agg([np.mean])
df_Sb_norm_sub_mean["sample"] = df_Sb_norm[~df_Sb_norm.index.duplicated(keep='first')]["sample"]
df_Sb_norm_new_sub_mean = df_Sb_norm_new.drop("cycle", axis=1).groupby(df_Sb_norm_new.index).agg([np.mean])
df_Sb_norm_new_sub_mean["sample"] = df_Sb_norm_new[~df_Sb_norm_new.index.duplicated(keep='first')]["sample"]
df_Sb_norm_new_117_sub_mean = df_Sb_norm_new_117.drop("cycle", axis=1).groupby(df_Sb_norm_new_117.index).agg([np.mean])
df_Sb_norm_new_117_sub_mean["sample"] = df_Sb_norm_new_117[~df_Sb_norm_new_117.index.duplicated(keep='first')]["sample"]
df_internal_norm_sub_mean = df_internal_norm.drop("cycle", axis=1).groupby(df_internal_norm.index).agg([np.mean])
df_internal_norm_sub_mean["sample"] = df_internal_norm[~df_internal_norm.index.duplicated(keep='first')]["sample"]
df_int_norm_new_117_sub_mean = df_int_norm_new_117.drop("cycle", axis=1).groupby(df_int_norm_new_117.index).agg([np.mean])
df_int_norm_new_117_sub_mean["sample"] = df_int_norm_new_117[~df_int_norm_new_117.index.duplicated(keep='first')]["sample"]

df_Sb_norm_sub_mean_sd = df_Sb_norm.drop("cycle", axis=1).groupby(df_Sb_norm.index).agg([np.mean, SD2])
df_Sb_norm_sub_mean_sd["sample"] = df_Sb_norm[~df_Sb_norm.index.duplicated(keep='first')]["sample"]
df_Sb_norm_new_sub_mean_sd = df_Sb_norm_new.drop("cycle", axis=1).groupby(df_Sb_norm_new.index).agg([np.mean, SD2])
df_Sb_norm_new_sub_mean_sd["sample"] = df_Sb_norm_new[~df_Sb_norm_new.index.duplicated(keep='first')]["sample"]
df_Sb_norm_new_117_sub_mean_sd = df_Sb_norm_new_117.drop("cycle", axis=1).groupby(df_Sb_norm_new_117.index).agg([np.mean, SD2])
df_Sb_norm_new_117_sub_mean_sd["sample"] = df_Sb_norm_new_117[~df_Sb_norm_new_117.index.duplicated(keep='first')]["sample"]
df_internal_norm_sub_mean_sd = df_internal_norm.drop("cycle", axis=1).groupby(df_internal_norm.index).agg([np.mean, SD2])
df_internal_norm_sub_mean_sd["sample"] = df_internal_norm[~df_internal_norm.index.duplicated(keep='first')]["sample"]
df_int_norm_new_117_sub_mean_sd = df_int_norm_new_117.drop("cycle", axis=1).groupby(df_int_norm_new_117.index).agg([np.mean, SD2])
df_int_norm_new_117_sub_mean_sd["sample"] = df_int_norm_new_117[~df_int_norm_new_117.index.duplicated(keep='first')]["sample"]

In [ ]:
df_Sb_norm_sub_mean_sd

In [ ]:
df_Sb_norm_new_sub_mean_sd

In [ ]:
df_internal_norm_sub_mean_sd

In [ ]:
df_Sb_norm_new_117_sub_mean_sd

In [ ]:
df_int_norm_new_117_sub_mean_sd

In [ ]:
df_Sb_norm_sub_sample_sd = df_Sb_norm_sub_mean[df_Sb_norm_sub_mean.index >= "2017-09-15 12:53:00"].groupby(df_Sb_norm_sub_mean["sample"]).agg([np.mean, SD2])
df_Sb_norm_sub_sample_sd

In [ ]:
df_Sb_norm_new_sub_sample_sd = df_Sb_norm_new_sub_mean[df_Sb_norm_new_sub_mean.index >= "2017-09-15 12:53:00"].groupby(df_Sb_norm_new_sub_mean["sample"]).agg([np.mean, SD2])
df_Sb_norm_new_sub_sample_sd

In [ ]:
df_internal_norm_sample_sd = df_internal_norm_sub_mean[df_internal_norm_sub_mean.index >= "2017-09-15 12:53:00"].groupby(df_internal_norm_sub_mean["sample"]).agg([np.mean, SD2])
df_internal_norm_sample_sd

In [ ]:
df_Sb_norm_new_117_sub_sample_sd = df_Sb_norm_new_117_sub_mean[df_Sb_norm_new_117_sub_mean.index >= "2017-09-15 12:53:00"].groupby(df_Sb_norm_new_sub_mean["sample"]).agg([np.mean, SD2])
df_Sb_norm_new_117_sub_sample_sd

In [ ]:
df_Sb_norm_sub_mean_sd.to_csv(path+"offline_Sb_norm.csv")

In [ ]:
df_Sb_norm_new_sub_mean_sd.to_csv(path+"offline_Sb_norm_Sb_ratio_altered_0.747491.csv")

In [ ]:
df_internal_norm_sub_mean_sd.to_csv(path+"offline_int_norm.csv")

In [ ]:
df_Sb_norm_new_117_sub_mean_sd.to_csv(path+"offline_Sb_norm_Sb_ratio_altered_117_0.747491.csv")

In [ ]:
df_int_norm_new_117_sub_mean_sd.to_csv(path+"offline_int_norm_122_117_altered.csv")

In [ ]:
df_all.to_csv(path+"15.09_Signals_zero_corr.csv")